In [1]:
! pip install kaggle

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105797 sha256=5756f442f54491b7f95d83bd87bc5608f3f4c92f975f99843db25a89ce711595
  Stored in directory: c:\users\matth\appdata\local\pip\cache\wheels\ff\55\fb\b27a466be754d2a06ffe0e37b248d844f090a63b51becea85d
Successfully built kaggle


In [6]:

from zipfile import ZipFile
import pandas as pd

Upload your Kaggle.json file


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

La syntaxe de la commande n'est pas correcte.
'cp' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'chmod' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Importing Sentiment dataset

In [3]:
#!/bin/bash
!kaggle datasets download hbaflast/french-twitter-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/hbaflast/french-twitter-sentiment-analysis
License(s): DbCL-1.0




  0%|          | 0.00/48.3M [00:00<?, ?B/s]
  2%|▏         | 1.00M/48.3M [00:00<00:38, 1.30MB/s]
  4%|▍         | 2.00M/48.3M [00:01<00:20, 2.32MB/s]
  6%|▌         | 3.00M/48.3M [00:01<00:21, 2.23MB/s]
  8%|▊         | 4.00M/48.3M [00:01<00:18, 2.51MB/s]
 10%|█         | 5.00M/48.3M [00:02<00:15, 2.91MB/s]
 12%|█▏        | 6.00M/48.3M [00:02<00:20, 2.14MB/s]
 14%|█▍        | 7.00M/48.3M [00:03<00:17, 2.53MB/s]
 17%|█▋        | 8.00M/48.3M [00:04<00:24, 1.72MB/s]
 19%|█▊        | 9.00M/48.3M [00:05<00:29, 1.38MB/s]
 21%|██        | 10.0M/48.3M [00:05<00:26, 1.52MB/s]
 23%|██▎       | 11.0M/48.3M [00:06<00:22, 1.75MB/s]
 25%|██▍       | 12.0M/48.3M [00:06<00:19, 1.95MB/s]
 27%|██▋       | 13.0M/48.3M [00:07<00:18, 1.95MB/s]
 29%|██▉       | 14.0M/48.3M [00:07<00:18, 1.93MB/s]
 31%|███       | 15.0M/48.3M [00:08<00:16, 2.07MB/s]
 33%|███▎      | 16.0M/48.3M [00:08<00:16, 2.11MB/s]
 35%|███▌      | 17.0M/48.3M [00:09<00:22, 1.43MB/s]
 37%|███▋      | 18.0M/48.3M [00:10<00:19, 1.65MB/s]
 

In [8]:
# extracting the compressed dataset (csv inside the zip file called french_tweets.csv)

dataset = 'french-twitter-sentiment-analysis.zip' # path of the dataset

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Data checkings

In [9]:
# loading the data from the csv file to pandas dataframe
twitter_data = pd.read_csv('french_tweets.csv')

In [ ]:
# checking number of rows and columns

twitter_data.shape

(1526724, 2)

In [ ]:
twitter_data.head()

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


In [ ]:
# counting the number of missingvalues
twitter_data.isnull().sum()

,0
label,0
text,0


In [ ]:
# checking the distribution of target column
twitter_data['label'].value_counts()

,count
label,
0,771604
1,755120
